
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

## OPEN PAYMENTS DATA

In [0]:
#unzipping the open payments zip file
dbutils.fs.cp("dbfs:/FileStore/tables/PGYR2023_P01302025_01212025.zip", "file:/tmp/PGYR2023_P01302025.zip")
import zipfile

with zipfile.ZipFile("/tmp/PGYR2023_P01302025.zip", "r") as zip_ref:
    zip_ref.extractall("/tmp/")

# List extracted files
import os
os.listdir("/tmp/") #ensuring the zip file is unzipped properly

Out[2]: ['hsperfdata_root',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv',
 'Rserv',
 'Rtmp4oxR6z',
 '.font-unix',
 'chauffeur-daemon-params',
 '.X11-unix',
 'OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv',
 '.ICE-unix',
 '.Test-unix',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-systemd-resolved.service-JHfnpi',
 'OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv',
 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-apache2.service-2mJw2i',
 'driver-daemon-params',
 'PGYR2023_P01302025.zip',
 '.XIM-unix',
 'OP_PGYR2023_README_P01302025.txt',
 'tmp.jbElhfi48u',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-ntp.service-TCyRWg',
 'chauffeur-env.sh',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-systemd-logind.service-Uio3hf',
 'chauffeur-daemon.pid',
 'custom-spark.conf',
 '.PGYR2023_P01302025.zip.crc',
 'driver-daemon.pid',
 'driver-env.sh']

In [0]:
#Moving the csv file back to the DBFS
dbutils.fs.mv(f"file:/tmp/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", 
              "dbfs:/FileStore/tables/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv")

Out[3]: True

In [0]:
display(dbutils.fs.ls("/FileStore/tables/"))

path,name,size,modificationTime
dbfs:/FileStore/tables/Drug_Interval-1.txt,Drug_Interval-1.txt,716,1742333455000
dbfs:/FileStore/tables/Drug_Interval.txt,Drug_Interval.txt,716,1742333124000
dbfs:/FileStore/tables/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv,OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv,357419225,1742683252000
dbfs:/FileStore/tables/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv,OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv,8159850128,1742750470000
dbfs:/FileStore/tables/OP_DTL_GNRL_PGYR2023_P01302025_01212025.zip,OP_DTL_GNRL_PGYR2023_P01302025_01212025.zip,725603560,1742662747000
dbfs:/FileStore/tables/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025-1.csv,OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025-1.csv,1827342,1742493341000
dbfs:/FileStore/tables/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv,OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv,1827342,1742445454000
dbfs:/FileStore/tables/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv,OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv,992593986,1742487750000
dbfs:/FileStore/tables/OP_PGYR2023_README_P01302025.txt,OP_PGYR2023_README_P01302025.txt,5422,1742445558000
dbfs:/FileStore/tables/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv,OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv,158460,1742445460000


In [0]:
# File location and type
file_location = "/FileStore/tables/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
open_payments_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
display(open_payments_df.limit(10))

Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_Profile_ID,Covered_Recipient_NPI,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,Covered_Recipient_Name_Suffix,Recipient_Primary_Business_Street_Address_Line1,Recipient_Primary_Business_Street_Address_Line2,Recipient_City,Recipient_State,Recipient_Zip_Code,Recipient_Country,Recipient_Province,Recipient_Postal_Code,Covered_Recipient_Primary_Type_1,Covered_Recipient_Primary_Type_2,Covered_Recipient_Primary_Type_3,Covered_Recipient_Primary_Type_4,Covered_Recipient_Primary_Type_5,Covered_Recipient_Primary_Type_6,Covered_Recipient_Specialty_1,Covered_Recipient_Specialty_2,Covered_Recipient_Specialty_3,Covered_Recipient_Specialty_4,Covered_Recipient_Specialty_5,Covered_Recipient_Specialty_6,Covered_Recipient_License_State_code1,Covered_Recipient_License_State_code2,Covered_Recipient_License_State_code3,Covered_Recipient_License_State_code4,Covered_Recipient_License_State_code5,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Physician_Ownership_Indicator,Third_Party_Payment_Recipient_Indicator,Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value,Charity_Indicator,Third_Party_Equals_Covered_Recipient_Indicator,Contextual_Information,Delay_in_Publication_Indicator,Record_ID,Dispute_Status_for_Publication,Related_Product_Indicator,Covered_or_Noncovered_Indicator_1,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1,Product_Category_or_Therapeutic_Area_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Associated_Drug_or_Biological_NDC_1,Associated_Device_or_Medical_Supply_PDI_1,Covered_or_Noncovered_Indicator_2,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2,Product_Category_or_Therapeutic_Area_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Associated_Drug_or_Biological_NDC_2,Associated_Device_or_Medical_Supply_PDI_2,Covered_or_Noncovered_Indicator_3,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3,Product_Category_or_Therapeutic_Area_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Associated_Drug_or_Biological_NDC_3,Associated_Device_or_Medical_Supply_PDI_3,Covered_or_Noncovered_Indicator_4,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Associated_Device_or_Medical_Supply_PDI_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Associated_Device_or_Medical_Supply_PDI_5,Program_Year,Payment_Publication_Date
CHANGED,Covered Recipient Teaching Hospital,220162,9984,Adventhealth Orlando,null,null,null,null,null,null,601 E Rollins St,null,Orlando,FL,32803,United States,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Integra LifeSciences Corporation,100000010950,Integra LifeSciences Corporation,NJ,United States,2000.00,07/20/2023,1,Cash or cash equivalent,Grant,null,null,null,No,No Third Party Payment,null,null,null,null,No,1031636873,No,Yes,Covered,Device,Neuro Critical Care,CODMAN CERTAS,null,10381780529033,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2023,01/30/2025
CHANGED,Covered Recipient Teaching Hospital,220162,9

## COVERED RECEIPIENTS DATA

In [0]:
#unzipping the open payments zip file
dbutils.fs.cp("dbfs:/FileStore/tables/PHPRFL_P01302025_01212025.zip", "file:/tmp/PHPRFL_P01302025.zip")

with zipfile.ZipFile("/tmp/PHPRFL_P01302025.zip", "r") as zip_ref:
    zip_ref.extractall("/tmp/")

# List extracted files
os.listdir("/tmp/") #ensuring the zip file is unzipped properly

Out[6]: ['hsperfdata_root',
 'OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv',
 'Rserv',
 'Rtmp4oxR6z',
 '.font-unix',
 'chauffeur-daemon-params',
 '.X11-unix',
 'OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv',
 'OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv',
 '.ICE-unix',
 '.Test-unix',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-systemd-resolved.service-JHfnpi',
 'OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv',
 '.PHPRFL_P01302025.zip.crc',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-apache2.service-2mJw2i',
 'driver-daemon-params',
 'OP_CVRD_RCPNT_PRFL_SPLMTL_README_P01302025.txt',
 'PGYR2023_P01302025.zip',
 '.XIM-unix',
 'OP_PGYR2023_README_P01302025.txt',
 'tmp.jbElhfi48u',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-ntp.service-TCyRWg',
 'chauffeur-env.sh',
 'systemd-private-c7f4359a1d6342a09bc2ccb2eb95edb7-systemd-logind.service-Uio3hf',
 'chauffeur-daemon.pid',
 'custom-spark.conf',
 '.PGYR2023_P01302025.zip.crc',
 'driver-daemon.pid',
 'driver-env

In [0]:
#Moving the csv file back to the DBFS
dbutils.fs.mv(f"file:/tmp/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv", 
              "dbfs:/FileStore/tables/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv")

Out[7]: True

In [0]:
# File location and type
file_location = "/FileStore/tables/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
covered_receip_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(covered_receip_df.limit(10))

Covered_Recipient_Profile_Type,Covered_Recipient_Profile_ID,Associated_Covered_Recipient_Profile_ID_1,Associated_Covered_Recipient_Profile_ID_2,Covered_Recipient_NPI,Covered_Recipient_Profile_First_Name,Covered_Recipient_Profile_Middle_Name,Covered_Recipient_Profile_Last_Name,Covered_Recipient_Profile_Suffix,Covered_Recipient_Profile_Alternate_First_Name,Covered_Recipient_Profile_Alternate_Middle_Name,Covered_Recipient_Profile_Alternate_Last_Name,Covered_Recipient_Profile_Alternate_Suffix,Covered_Recipient_Profile_Address_Line_1,Covered_Recipient_Profile_Address_Line_2,Covered_Recipient_Profile_City,Covered_Recipient_Profile_State,Covered_Recipient_Profile_Zipcode,Covered_Recipient_Profile_Country_Name,Covered_Recipient_Profile_Province_Name,Covered_Recipient_Profile_Primary_Specialty,Covered_Recipient_Profile_OPS_Taxonomy_1,Covered_Recipient_Profile_OPS_Taxonomy_2,Covered_Recipient_Profile_OPS_Taxonomy_3,Covered_Recipient_Profile_OPS_Taxonomy_4,Covered_Recipient_Profile_OPS_Taxonomy_5,Covered_Recipient_Profile_OPS_Taxonomy_6,Covered_Recipient_Profile_License_State_Code_1,Covered_Recipient_Profile_License_State_Code_2,Covered_Recipient_Profile_License_State_Code_3,Covered_Recipient_Profile_License_State_Code_4,Covered_Recipient_Profile_License_State_Code_5
Covered Recipient Physician,1,null,null,1003020595,BARRY,D,SIMMONS,null,BARRON,DAVID,SIMMONS,null,847 SOUTH MILLEDGE AVE,null,ATHENS,GA,30605-1331,UNITED STATES,null,Dental Providers|Dentist|General Practice,1223G0001X,null,null,null,null,null,GA,null,null,null,null
Covered Recipient Physician,2,null,null,1003022823,LINDSEY,R,SCHUSTER,null,LINDSEY,R,GROBER,null,15 HEIDI LN,null,MOUNT SINAI,NY,11766-1428,UNITED STATES,null,Allopathic & Osteopathic Physicians|Emergency Medicine,207P00000X,null,null,null,null,null,NY,PA,null,null,null
Covered Recipient Physician,3,null,null,1003023359,JUSTIN,H,RACKLEY,null,JUSTIN,HAMWAY,RACKLEY,null,1001 SAM PERRY BLVD,null,FREDERICKSBURG,VA,22401-4453,UNITED STATES,null,Allopathic & Osteopathic Physicians|Anesthesiology,207L00000X,null,null,null,null,null,VA,OH,null,null,null
Covered Recipient Physician,4,null,null,1003024811,LISA,L,HAMAKER,null,null,null,null,null,100 MARKET ST STE 300,null,COLLEGEVILLE,PA,19426-4927,UNITED STATES,null,"Allopathic & Osteopathic Physicians|Internal Medicine|Endocrinology, Diabetes & Metabolism",207RE0101X,null,null,null,null,null,PA,null,null,null,null
Covered Recipient Physician,5,null,null,1003025875,PATRICK,E,DAVOL,null,PATRICK,ERIN,DAVOL,null,1698 E MCANDREWS RD STE 280,null,MEDFORD,OR,97504-5590,UNITED STATES,null,Allopathic & Osteopathic Physicians|Urology,208800000X,null,null,null,null,null,OR,IN,null,null,null
Covered Recipient Physician,6,null,null,1003026410,VIJAYA,L,MUMMADI,null,VIJAYA,null,MUMMADI,null,9900 N CENTRAL EXPY STE 225,null,DALLAS,TX,75231-0918,UNITED STATES,null,Allopathic & Osteopathic Physicians|Internal Medicine,207R00000X,null,null,null,null,null,TX,MI,FL,IL,null
Covered Recipient Physician,7,null,null,1003028150,JILL,null,WIRTH RISSMAN,null,JILL,null,WIRTH-RISSMAN,null,205 N CUMMINGS LN,null,WASHINGTON,IL,61571-2181,UNITED STATES,null,Allopathic & Osteopathic Physicians|Family Medicine|Sports Medicine,207QS0010X,207Q00000X,null,null,null,null,IL,MO,null,null,null
Covered Recipient Physician,9,null,null,1003033051,MARIANNA,A,POST,null,MARIANNA,ALEXSANDRA,POST,null,701 W COCOA BEACH CSWY,null,COCOA BEACH,FL,32931,UNITED STATES,null,Allopathic & Osteopathic Physicians|Hospitalist,207Q00000X,208M00000X,null,null,null,null,FL,MI,null,null,null
Covered Recipient Physician,10,null,null,1003000407,DAVID,J,GIRARDI,null,DAVID,JAMES,GIRARDI,null,306 PENN AVE,null,PITTSBURGH,PA,15221-2134,UNITED STATES,null,Allopathic & Osteopathic Physicians|Family Medicine,207Q00000X,208M00000X,208D00000X,null,null,null,PA,null,null,null,null
Covered Recipient Physician,11,null,null,1003002072,ALEXANDER,null,GOTESMAN,null,null,null,null,null,1374 WHITEHORSE HAMILTON SQUARE RD,STE 101,HAMILTON,NJ,08690-3701,U

## MERGE BOTH FILES

In [0]:
#merging both the files on covered_recipient_profile_ID(common in both the datasets)
from pyspark.sql.functions import col, sum, count, desc #importing the sql functions for running the below queries
final_df = open_payments_df.join(covered_receip_df, on="Covered_Recipient_Profile_ID", how="inner")
display(final_df.limit(10))

Covered_Recipient_Profile_ID,Change_Type,Covered_Recipient_Type,Teaching_Hospital_CCN,Teaching_Hospital_ID,Teaching_Hospital_Name,Covered_Recipient_NPI,Covered_Recipient_First_Name,Covered_Recipient_Middle_Name,Covered_Recipient_Last_Name,Covered_Recipient_Name_Suffix,Recipient_Primary_Business_Street_Address_Line1,Recipient_Primary_Business_Street_Address_Line2,Recipient_City,Recipient_State,Recipient_Zip_Code,Recipient_Country,Recipient_Province,Recipient_Postal_Code,Covered_Recipient_Primary_Type_1,Covered_Recipient_Primary_Type_2,Covered_Recipient_Primary_Type_3,Covered_Recipient_Primary_Type_4,Covered_Recipient_Primary_Type_5,Covered_Recipient_Primary_Type_6,Covered_Recipient_Specialty_1,Covered_Recipient_Specialty_2,Covered_Recipient_Specialty_3,Covered_Recipient_Specialty_4,Covered_Recipient_Specialty_5,Covered_Recipient_Specialty_6,Covered_Recipient_License_State_code1,Covered_Recipient_License_State_code2,Covered_Recipient_License_State_code3,Covered_Recipient_License_State_code4,Covered_Recipient_License_State_code5,Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Total_Amount_of_Payment_USDollars,Date_of_Payment,Number_of_Payments_Included_in_Total_Amount,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Physician_Ownership_Indicator,Third_Party_Payment_Recipient_Indicator,Name_of_Third_Party_Entity_Receiving_Payment_or_Transfer_of_Value,Charity_Indicator,Third_Party_Equals_Covered_Recipient_Indicator,Contextual_Information,Delay_in_Publication_Indicator,Record_ID,Dispute_Status_for_Publication,Related_Product_Indicator,Covered_or_Noncovered_Indicator_1,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1,Product_Category_or_Therapeutic_Area_1,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1,Associated_Drug_or_Biological_NDC_1,Associated_Device_or_Medical_Supply_PDI_1,Covered_or_Noncovered_Indicator_2,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_2,Product_Category_or_Therapeutic_Area_2,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2,Associated_Drug_or_Biological_NDC_2,Associated_Device_or_Medical_Supply_PDI_2,Covered_or_Noncovered_Indicator_3,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_3,Product_Category_or_Therapeutic_Area_3,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3,Associated_Drug_or_Biological_NDC_3,Associated_Device_or_Medical_Supply_PDI_3,Covered_or_Noncovered_Indicator_4,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4,Product_Category_or_Therapeutic_Area_4,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4,Associated_Drug_or_Biological_NDC_4,Associated_Device_or_Medical_Supply_PDI_4,Covered_or_Noncovered_Indicator_5,Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5,Product_Category_or_Therapeutic_Area_5,Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5,Associated_Drug_or_Biological_NDC_5,Associated_Device_or_Medical_Supply_PDI_5,Program_Year,Payment_Publication_Date,Covered_Recipient_Profile_Type,Associated_Covered_Recipient_Profile_ID_1,Associated_Covered_Recipient_Profile_ID_2,Covered_Recipient_NPI,Covered_Recipient_Profile_First_Name,Covered_Recipient_Profile_Middle_Name,Covered_Recipient_Profile_Last_Name,Covered_Recipient_Profile_Suffix,Covered_Recipient_Profile_Alternate_First_Name,Covered_Recipient_Profile_Alternate_Middle_Name,Covered_Recipient_Profile_Alternate_Last_Name,Covered_Recipient_Profile_Alternate_Suffix,Covered_Recipient_Profile_Address_Line_1,Covered_Recipient_Profile_Address_Line_2,Covered_Recipient_Profile_City,Covered_Recipient_Profile_State,Covered_Recipient_Profile_Zipcode,Covered_Recipient_Profile_Country_Name,Covered_Recipient_Profile_Province_Name,Covered_Recipient_

### 1. What is the Nature of Payments with reimbursement amounts greater than $1,000 ordered by count?

To identify the most common natures of payment for transactions where the reimbursement amount is greater than 1000 dollars, ranking them by their frequency, firstly only the rows where the total payment amount exceeds $1000 are retained. Next, aggregated the payments based on their nature of payment, meaning all similar types of payments are grouped together. Next, how many times each type of payment appears in the filtered dataset are assigned as "count". Finally, the results from most frequent to least frequent nature of payment are shown.

In [0]:
high_reimb_payment_nature = (final_df.filter(col("Total_Amount_of_Payment_USDollars") > 1000)
    .groupBy("Nature_of_Payment_or_Transfer_of_Value")
    .agg(count("Nature_of_Payment_or_Transfer_of_Value").alias("count"))
    .orderBy(desc("count")))
display(high_reimb_payment_nature.limit(10))

Nature_of_Payment_or_Transfer_of_Value,count
"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",161476
Consulting Fee,104564
Travel and Lodging,24711
Honoraria,13741
Education,12503
Royalty or License,10573
Compensation for serving as faculty or as a speaker for a medical education program,8655
Grant,2222
Long term medical supply or device loan,998
Food and Beverage,951


The top 3 nature of payments having >$1000 reimbursement ordered by count are found to be:
1. Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program
2. Consulting fee
3. Travel and Lodging

### 2. What are the top ten Nature of Payments by count?

To identify the most common types of payments or transfers of value by counting their occurrences in the dataset, the merged data is initially grouped by the "Nature_of_Payment_or_Transfer_of_Value" column, meaning that all rows with the same nature of payment are combined. Then, count("Nature_of_Payment_or_Transfer_of_Value") function counts how many times each nature of payment appears in the dataset. The resulting count is assigned the alias "count". The results are ordered in descending order of "count", meaning the most frequently occurring nature of payment appears at the top. The display(top_nature_by_count.limit(10)) function ensures that only the top 10 most frequently occurring payment types are shown.

In [0]:
top_nature_by_count = (final_df.groupBy("Nature_of_Payment_or_Transfer_of_Value")
    .agg(count("Nature_of_Payment_or_Transfer_of_Value").alias("count"))
    .orderBy(desc("count")))
display(top_nature_by_count.limit(10))

Nature_of_Payment_or_Transfer_of_Value,count
Food and Beverage,13378081
Travel and Lodging,545048
"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",230121
Consulting Fee,169540
Education,159397
Gift,31695
Honoraria,20214
Royalty or License,14007
Compensation for serving as faculty or as a speaker for a medical education program,12231
Entertainment,7967


The top 10 nature of payments by count are found to be:
1. Food and Beverage
2. Travel and Lodging
3. Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program
4. Consulting Fee
5. Education
6. Gift
7. Honoraria
8. Royalty or License
9. Compensation for serving as faculty or as a speaker for a medical education program
10. Entertainment

### 3. What are the top ten Nature of Payments by total amount?

To identify the top natures of payment based on the total amount paid rather than just their count, the dataset is first grouped by "Nature_of_Payment_or_Transfer_of_Value", meaning all rows with the same nature of payment are combined.Then, the su ("Total_Amount_of_Payment_USDollars") function calculates the total amount paid for each nature of payment. Next, the result is assigned alias "total_amount". Then, results are ordered in descending order of "total_amount", so the payment types with the highest total payment amounts appear at the top. Finally, display(top_nature_by_amount.limit(10)) function ensures that only the top 10 natures of payment with the highest total payment amounts are shown.

In [0]:
top_nature_by_amount = (final_df.groupBy("Nature_of_Payment_or_Transfer_of_Value")
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount"))
    .orderBy(desc("total_amount")))
display(top_nature_by_amount.limit(10))

Nature_of_Payment_or_Transfer_of_Value,total_amount
Royalty or License,6.242166887899998E8
"Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program",5.523960288000001E8
Consulting Fee,4.988225872099996E8
Food and Beverage,3.743433390699896E8
Travel and Lodging,1.7943128267000863E8
Acquisitions,7.192577675999999E7
Education,5.541025349000224E7
Honoraria,5.523104170999996E7
Compensation for serving as faculty or as a speaker for a medical education program,2.8615945760000013E7
Grant,2.4814728549999997E7


The top 10 nature of payments by total amount are found to be:
1. Royalty or License
2. Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program
3. Consulting Fee
4. Food and Beverage
5. Travel and Lodging
6. Acquisitions
7. Education
8. Honoraria
9. Compensation for serving as faculty or as a speaker for a medical education program
10. Grant

According to the txt files provided with the zip files, the covered recepients include MSC333", non-Physician practitioners,and physicians so, as per the question the data is filtered here to include only physicians data to answer the below questions.

In [0]:
display(final_df.select("Covered_Recipient_Type").distinct())

Covered_Recipient_Type
Covered Recipient Non-Physician Practitioner
Covered Recipient Physician
"MSC333"""


In [0]:
#filtering to include only physician data
final_df = final_df.filter(col("Covered_Recipient_Type") == "Covered Recipient Physician")

### 4. What are the top ten physician specialties by total amount?

To identify the top medical specialties based on the total amount of payments received, first the dataset is grouped by "Covered_Recipient_Profile_Primary_Specialty", meaning all payments related to the same specialty are combined. Next, the sum("Total_Amount_of_Payment_USDollars") function calculates the total amount received for each specialty. The result is assigned as "total_amount". Finally,the results are ordered in descending order of "total_amount", so the specialties receiving the highest total payments appear at the top. The display(top_specialties_by_amount.limit(10)) function ensures that only the top 10 specialties by total payment amount are shown.

In [0]:
top_specialties_by_amount = (final_df.groupBy("Covered_Recipient_Profile_Primary_Specialty")
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount"))
    .orderBy(desc("total_amount")))
display(top_specialties_by_amount.limit(10))

Covered_Recipient_Profile_Primary_Specialty,total_amount
Allopathic & Osteopathic Physicians|Orthopaedic Surgery,3.777151620799968E8
Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology,8.667850010000029E7
Allopathic & Osteopathic Physicians|Neurological Surgery,8.586441204999988E7
Allopathic & Osteopathic Physicians|Dermatology,7.997300741000053E7
Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Adult Reconstructive Orthopaedic Surgery,7.277526618000007E7
Allopathic & Osteopathic Physicians|Internal Medicine|Hematology & Oncology,6.567412825000092E7
Allopathic & Osteopathic Physicians|Internal Medicine,6.555511652000128E7
Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Psychiatry,6.527284579000072E7
Allopathic & Osteopathic Physicians|Internal Medicine|Cardiovascular Disease,6.29944522400002E7
Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Orthopaedic Surgery of the Spine,6.143177107000011E7


The top 10 physician specialties by total amount are found to be:
1. Allopathic & Osteopathic Physicians|Orthopaedic Surgery
2. Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology
3. Allopathic & Osteopathic Physicians|Neurological Surgery
4. Allopathic & Osteopathic Physicians|Dermatology
5. Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Adult Reconstructive Orthopaedic Surgery
6. Allopathic & Osteopathic Physicians|Internal Medicine|Hematology & Oncology
7. Allopathic & Osteopathic Physicians|Internal Medicine
8. Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Psychiatry
9. Allopathic & Osteopathic Physicians|Internal Medicine|Cardiovascular Disease
10. Allopathic & Osteopathic Physicians|Orthopaedic Surgery|Orthopaedic Surgery of the Spine


### 5. Who are the top ten physicians by total amount?

To identify the top physicians who have received the highest total payments, firstly, the query replaces any NULL values in the "Covered_Recipient_Profile_Middle_Name" column with an empty string ("") using coalesce(). This ensures that physicians with missing middle names are still properly grouped without errors. The data is then grouped by first name, middle name, and last name to uniquely identify physicians. Next, the query calculates the total amount of payments (in USD) for each physician using the sum() function and the aggregated column is named "total_amount". The results are finally sorted in descending order based on "total_amount", so the physicians who received the highest payments appear first.

In [0]:
from pyspark.sql.functions import coalesce, lit
top_physicians_by_amount = (final_df.withColumn("Covered_Recipient_Profile_Middle_Name", 
                        coalesce(col("Covered_Recipient_Profile_Middle_Name"), lit(""))) #Replace Null with empty string
    .groupBy("Covered_Recipient_Profile_First_Name", 
             "Covered_Recipient_Profile_Middle_Name", "Covered_Recipient_Profile_Last_Name")
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount"))
    .orderBy(desc("total_amount")))
display(top_physicians_by_amount.limit(10))

Covered_Recipient_Profile_First_Name,Covered_Recipient_Profile_Middle_Name,Covered_Recipient_Profile_Last_Name,total_amount
STEPHEN,S,BURKHART,3.392202493E7
WILLIAM,JAY,BINDER,2.943435593E7
KEVIN,T,FOLEY,1.730721365E7
IVAN,,OSORIO,1.606551551E7
GEORGE,PATRICK,MAXWELL,1.160032024E7
ROBERT,E,BOOTH,8459144.4
NEAL,S,ELATTRACHE,7810628.199999999
AARON,,ROSENBERG,6871466.720000001
ROGER,P,JACKSON,6660383.800000001
PETER,M,BONUTTI,6385096.170000002


The top 10 physicians by total amount are found to be:
1. Stephen S Burkhart
2. William Jay Binder
3. Kevin T Foley
4. Ivan Osorio
5. George Patrick Maxwell
6. Robert E Booth
7. Neal S Elattrache
8. Aaron Rosenberg
9. Roger P Jackson
10. Peter M Bonutti